In [3]:
# 1. 필요한 라이브러리 임포트
import time
from datetime import datetime
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests

In [ ]:
# # 2. 웹드라이버 설정
# def setup_driver():
#     chrome_options = Options()
#     chrome_options.add_argument('--disable-blink-features=AutomationControlled')
#     chrome_options.add_argument('--disable-gpu')
#     chrome_options.add_argument('--no-sandbox')
#     chrome_options.add_argument('--disable-dev-shm-usage')
#     chrome_options.add_argument('--window-size=1920,1080')
#     chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
#                             options=chrome_options)
#     return driver


In [8]:
# 2. 웹드라이버 설정
def setup_driver():
    chrome_options = Options()
    
    # 리눅스 환경에서의 추가 설정
    chrome_options.add_argument('--headless')  # 헤드리스 모드 실행
    chrome_options.add_argument('--no-sandbox')  # 샌드박스 비활성화
    chrome_options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 사용 비활성화
    chrome_options.add_argument('--disable-gpu')  # GPU 하드웨어 가속 비활성화
    
    # 기존 설정들
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    # 크롬 드라이버 생성 시 에러 처리
    try:
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        return driver
    except Exception as e:
        print(f"드라이버 설정 중 오류 발생: {e}")
        
        # 대체 방법 시도
        try:
            print("대체 방법으로 드라이버 설정 시도...")
            chrome_options.add_argument('--remote-debugging-port=9222')
            driver = webdriver.Chrome(
                options=chrome_options
            )
            return driver
        except Exception as sub_e:
            print(f"대체 방법도 실패: {sub_e}")
            raise

In [13]:
# 3. 공고 상세 페이지 크롤링 함수
def get_announcement_detail(url, headers):
    try:
        print("\n상세 페이지 데이터 수집 중...")
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        board_view = soup.find('div', class_='board_view')
        
        if not board_view:
            print("상세 페이지 구조 찾을 수 없음")
            return None
            
        data = {}
        
        # 제목
        title_tag = board_view.find('h4')
        data['title'] = title_tag.text.strip() if title_tag else ''
        
        # 테이블 데이터 추출
        table = board_view.find('table')
        if table:
            rows = table.find_all('tr', class_='web')
            for row in rows:
                ths = row.find_all('th')
                tds = row.find_all('td')
                for th, td in zip(ths, tds):
                    key = th.text.strip()
                    value = td.text.strip()
                    
                    if key == '공고번호':
                        data['category'] = value
                    elif key == '신청기간':
                        periods = value.split('~')
                        data['start_period'] = periods[0].strip()
                        data['end_period'] = periods[1].strip() if len(periods) > 1 else ''
                    elif key == '담당부서':
                        data['agency'] = value
                    elif key == '등록일':
                        data['post_date'] = value
        
        # 내용
        content_div = board_view.find('div', class_='txt-area')
        if content_div:
            data['content'] = content_div.text.strip()
        
        # 첨부파일 정보 추가
        files = []
        file_list = board_view.find('td', class_='file_list')
        if file_list:
            for file_item in file_list.find_all('li'):
                file_info = {}
                name_tag = file_item.find('span', class_='name')
                if name_tag:
                    file_info['name'] = name_tag.text.strip()
                
                download_link = file_item.find('a', class_='btn type_down')
                if download_link:
                    file_info['link'] = 'https://www.mss.go.kr' + download_link['href']
                
                files.append(file_info)
        
        data['attach_file'] = files
        data['link'] = url
        
        return data
        
    except Exception as e:
        print(f"상세 페이지 크롤링 중 오류 발생: {e}")
        return None

In [14]:
# 4. 목록 페이지 크롤링 함수
def get_announcement_list(base_url, start_page=1, end_page=1):
    announcements = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    try:
        for page in range(start_page, end_page + 1):
            print(f"\n{'='*50}")
            print(f"현재 {page}페이지 크롤링 시작")
            print(f"{'='*50}")
            
            url = f"{base_url}&pageIndex={page}"
            print(f"페이지 URL: {url}")
            
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # table 선택자 수정
            table = soup.find('table')
            if table and table.find('tbody'):
                rows = table.find('tbody').find_all('tr')  # tbody 내의 모든 tr 찾기
                print(f"\n총 {len(rows)}개의 공고 발견\n")
                
                for idx, row in enumerate(rows, 1):
                    print(f"\n--- 공고 {idx} 처리 중 ---")
                    try:
                        # 링크 추출 방식 수정
                        link_tag = row.find('td', class_='subject').find('a')
                        if link_tag:
                            title = link_tag.text.strip()
                            print(f"제목: {title}")
                            
                            onclick = link_tag.get('onclick', '')
                            match = re.search(r"doBbsFView\('(\d+)',\s*'(\d+)'", onclick)
                            if match:
                                cbIdx, bcIdx = match.groups()
                                detail_url = f"https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx={cbIdx}&bcIdx={bcIdx}&parentSeq={bcIdx}"
                                print(f"상세 페이지 URL: {detail_url}")
                                
                                announcement_data = get_announcement_detail(detail_url, headers)
                                if announcement_data:
                                    announcements.append(announcement_data)
                                    print("✓ 데이터 수집 성공")
                                else:
                                    print("✗ 데이터 수집 실패")
                    
                    except Exception as e:
                        print(f"공고 처리 중 오류: {e}")
                        continue
                        
                    time.sleep(1)
            
            print(f"\n{page}페이지 완료 - 총 {len(announcements)}개 수집됨")
            time.sleep(2)
            
    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")
    
    return announcements

In [15]:
# 실행
print("크롤링 시작...")
base_url = "https://www.mss.go.kr/site/smba/ex/bbs/List.do?cbIdx=310&searchRltnYn=A"
announcements = get_announcement_list(base_url, start_page=1, end_page=1)

print("\n최종 결과:")
print(f"총 {len(announcements)}개의 공고 수집 완료")

# 결과를 DataFrame으로 변환
df = pd.DataFrame(announcements)
display(df)

크롤링 시작...

현재 1페이지 크롤링 시작
페이지 URL: https://www.mss.go.kr/site/smba/ex/bbs/List.do?cbIdx=310&searchRltnYn=A&pageIndex=1

총 10개의 공고 발견


--- 공고 1 처리 중 ---
제목: 2025년 중소기업 혁신바우처 사업 지원계획 공고문
상세 페이지 URL: https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx=310&bcIdx=1054477&parentSeq=1054477

상세 페이지 데이터 수집 중...
✓ 데이터 수집 성공

--- 공고 2 처리 중 ---
제목: 소상공인 전기요금 특별지원사업 시행 수정공고
상세 페이지 URL: https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx=310&bcIdx=1054257&parentSeq=1054257

상세 페이지 데이터 수집 중...
✓ 데이터 수집 성공

--- 공고 3 처리 중 ---
제목: 벤처투자회사 등록말소
상세 페이지 URL: https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx=310&bcIdx=1054171&parentSeq=1054171

상세 페이지 데이터 수집 중...
✓ 데이터 수집 성공

--- 공고 4 처리 중 ---
제목: 2024년 소상공인 정책자금 융자계획 변경공고
상세 페이지 URL: https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx=310&bcIdx=1054113&parentSeq=1054113

상세 페이지 데이터 수집 중...
✓ 데이터 수집 성공

--- 공고 5 처리 중 ---
제목: 2024년 중소기업 정책자금 융자계획 변경공고
상세 페이지 URL: https://www.mss.go.kr/site/smba/ex/bbs/View.do?cbIdx=310&bcIdx=1054111&parentSeq=1054111


,title,category,start_period,end_period,agency,post_date,content,attach_file,link
0,2025년 중소기업 혁신바우처 사업 지원계획 공고문,제2024-567호,,,지역혁신정책과,2024.11.08,<br />\r\n중소벤처기업부 공고 제2024-567호<br />\r\n<br /...,[{'name': '2025년_중소기업_혁신바우처_사업_1차_지원계획_공고.hwpx...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
1,소상공인 전기요금 특별지원사업 시행 수정공고,제2024-557호,2024-11-01,,소상공인손실보상과,2024.11.01,2024년 소상공인 전기요금 특별지원 사업을 붙임과 같이 수정 공고합니다.&nbsp;,[{'name': '소상공인_전기요금_특별지원_사업_시행_수정공고(제2024-557...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
2,벤처투자회사 등록말소,중소벤처기업부 공고 제2024-554호,,,투자관리감독과,2024.10.29,중소벤처기업부 공고 제2024-554호<br />\r\n<br />\r\n벤처투자회...,[{'name': '중소벤처기업부_공고_제2024-554호(벤처투자회사_등록말소)....,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
3,2024년 소상공인 정책자금 융자계획 변경공고,제2024-551호,2024-10-28,,기업금융과,2024.10.25,<br />\r\n붙임과 같이 2024년 소상공인 정책자금 융자계획을 변경하여 공고...,[{'name': '2024년_소상공인_정책자금_운용계획(제2024-551호).hw...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
4,2024년 중소기업 정책자금 융자계획 변경공고,제2024-552호,,,기업금융과,2024.10.25,<br />\r\n2024년 중소기업 정책자금 융자계획을 붙임과 같이 변경 공고합니다.,[{'name': '2024년_중소기업_정책자금_융자계획_공고(제2024-552호)...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
5,(통합공고) 위메프·티몬·인터파크·AK몰·알렛츠 정산지연 피해 소상공인·중소기업에 ...,중소벤처기업부 공고 제2024-546호,2024-08-28,,디지털소상공인과,2024.10.24,ㅇ위메프&middot;티몬&middot;인터파크쇼핑&middot;AK몰&middot...,[{'name': '(통합공고)_위메프·티몬·인터파크·AK몰·알렛츠_정산지연_피해_...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
6,스케일업 팁스 운영사 모집 공고(연장),제2024-544호,2024-10-14,2024-11-01,기술혁신정책과,2024.10.24,"투자시장과 연구개발 분야의 민간 전문역량을 활용하여, 국가전략기술 등 주요 기술분야...",[{'name': '스케일업_팁스_운영사_모집_공고(연장).hwpx [1.3 MB...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
7,2024년 소상공인 정책자금 융자계획 변경공고,제2024-541호,2024-10-21,,기업금융과,2024.10.21,2024년 소상공인 정책자금 융자계획을 붙임과 같이 변경하여 공고합니다.&nbsp;,[{'name': '2024년_소상공인_정책자금_운용계획_변경공고(제2024-541...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
8,(수정 공고) 위메프·티몬·인터파크·AK몰·알렛츠 정산지연 피해 소상공인에 대한 이...,제2024 - 538호,2024-08-28,,디지털소상공인과,2024.10.17,"위메프, 티몬, 인터파크쇼핑, AK몰, 알렛츠의 판매대금 정산 지연으로 피해를 입은...",[{'name': '(수정_공고)_위메프·티몬·인터파크·AK몰·알렛츠_정산지연_피해...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
9,2024년 기술개발제품 시범구매 지원계획 공고(4차),제2024-536호,,,판로정책과,2024.10.16,<br />\r\n2024년 기술개발제품 시범구매 지원계획 공고(4차),[{'name': '(공고)_2024년_기술개발제품_시범구매_지원계획_공고(안)(4...,https://www.mss.go.kr/site/smba/ex/bbs/View.do...
